In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import json
import yaml
from collections import defaultdict

import pandas as pd
from recbole.config import Config
from recbole.data import data_preparation, create_dataset
from recbole.trainer import HyperTuning
from recbole.utils import (
    get_model,
    get_trainer,
    init_seed
)

import utils
from real_temporal import TimeCutoffDataset

# 1. Declarations & Definitions

## 1.1. Define flags and global variables

In [7]:
use_TimeCutoff = True
reproducible = True
seed = 42

## 1.2. Define configurations

Configuration for data, model, training and evaluation

In [8]:
def get_suitable_cutoff(ds_name: str) -> tuple:
    """Get suitable cutoff timestamp: at which there are the most active users

    Args:
        ds_name (str): dataset name

    Returns:
        tuple: suitable timestamp and the number of active users
    """

    # Get dataset without normalizing the timestamp
    config_dict = {
        'normalize_all': False,

        'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
        'train_neg_sample_args': None,

        'eval_args': {
            "order": "TO",
            "split": { "LS": "valid_and_test" },
            "group_by": None,
            'mode': 'full'
        },
    }
    config = Config(
        model='NPE',
        dataset=ds_name,
        config_dict=config_dict,
    )
    init_seed(config["seed"], config["reproducibility"])
    df = create_dataset(config).inter_feat.copy()

    # Create dataframe of users and corresponding first/last timestamp
    user_max_ts = df.groupby('user_id')['timestamp'].max()
    user_min_ts = df.groupby('user_id')['timestamp'].min()
    df_user = pd.DataFrame(
        {
            'max': user_max_ts,
            'min': user_min_ts,
        },
        index=user_max_ts.index
    )

    counts = defaultdict(int) 
    for ts in df_user['min']:
        counts[ts] += 1
    for ts in df_user['max']:
        counts[ts] -= 1

    timestamps = sorted(counts.keys())
    accum = {}

    s = 0
    for ts in timestamps:
        s += counts[ts]
        accum[ts] = s
    series = pd.Series(accum)

    suitable_ts = series.idxmax()
    max_active_user = series[suitable_ts]

    return suitable_ts, max_active_user

get_suitable_cutoff('ml-1M')

(976324045.0, 1190)

In [11]:
config_dict = {
    # For model
    'model': "Caser",
    'embedding_size': 64,
    'n_v': 4,
    'n_h': 8,
    'reg_weight': 1e-4,
    'dropout_prob': .4,
    'loss_type': "CE",

    # For data
    'dataset': 'ml-1M',
    'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
    'use_TimeCutoff': use_TimeCutoff,

    # For training
    'epochs': 500,
    'train_batch_size': 4096,
    'eval_step': 5,
    'stopping_step': 5,
    'learning_rate': 1e-3,
    'train_neg_sample_args': None,
    # 'train_neg_sample_args': {
    #     'distribution': 'uniform',
    #     'sample_num': 1,
    #     'dynamic':  True,
    #     'candidate_num': 0
    # },
    
    # For evaluation
    'eval_batch_size': 4096,
    'metrics': ["NDCG", "Precision", "Recall", "MRR", "Hit", "MAP"],
    'topk': 10,
    'valid_metric': 'NDCG@10',   # evaluation metric for earlystopping

    # Environment
    'checkpoint_dir': utils.get_path_dir_ckpt(),
    'device': 'mps',
    'show_progress': True,
    'reproducibility': reproducible,
    'seed': seed,
}

if use_TimeCutoff is True:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
             "split": {"CO": '976324045'},
            "group_by": 'user_id',
            'mode': 'full'
        },
    }
else:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
            "split": { "LS": "valid_and_test" },
            "group_by": None,
            'mode': 'full'
        },
    }

# 2. Train

## 2.1. Declare necessary components for training

In [12]:
config = Config(config_dict=config_dict)

init_seed(config["seed"], config["reproducibility"])

# Define data related things
if use_TimeCutoff:
    dataset = TimeCutoffDataset(config)
else:
    dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Define model
model_name = config['model']
model = get_model(model_name)(config, train_data._dataset).to(config['device'])

# Define trainer
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

In [13]:
logger = utils.get_logger()

print(" === Config === ")
logger.info(config)

print(" === Dataset === ")
logger.info(dataset)

print(" === Model === ")
logger.info(model)

2024-07-03 21:00:29.767 | INFO     | __main__:<module>:4 - 
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = dataset/ml-1M
checkpoint_dir = logs/Jul03_205409/ckpts
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 500
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 5
stopping_step = 5
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'CO': '976324045'}, 'order': 'TO', 'group_by': 'user_id', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['NDCG', 'Precision', 'Recall', 'MRR', 'Hit', 'MAP']
topk = [10]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size

 === Config === 
 === Dataset === 
 === Model === 


## 2.2. Start training

In [14]:
best_valid_score, best_valid_result = trainer.fit(
    train_data, 
    valid_data,
    verbose=True,
    show_progress=config["show_progress"]
)

print("** Validation result")
print(f"best_valid_score: {best_valid_score:.4f}")
for metric, val in best_valid_result.items():
    print(f"{metric:<15}: {val:.4f}")

Train     0:   7%|███▍                                           | 15/204 [04:48<1:00:34, 19.23s/it]


KeyboardInterrupt: 

## 3.3. Start testing

In [ ]:
test_result = trainer.evaluate(test_data)

print("** Test result")
for metric, val in test_result.items():
    print(f"{metric:<15}: {val:.4f}")

** Test result
ndcg@10        : 0.0642
precision@10   : 0.0120
recall@10      : 0.1198
mrr@10         : 0.0473
hit@10         : 0.1198
map@10         : 0.0473


# 4. Tune hyper params

## 4.1. Define hyper params and object function

In [ ]:
hyper_params = {
    # 'loguniform': {
    #     'learning_rate': [-8, 0]
    # },
    'choice': {
        # 'embedding_size': [64],
        # 'dropout_prob': [0.2, 0.3, .5]
        'learning_rate' : [0.01,0.005,0.001,0.0005,0.0001],
        # 'reg_weight' : [0,1e-4,1e-5],
        # 'dropout_prob' : [0.0,0.1,0.2,0.3,0.4,0.5]
    }
}

In [ ]:
def objective_function(config_dict=None, config_file_list=None):
    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
    )

    init_seed(config["seed"], config["reproducibility"])

    # Define data related things
    if use_TimeCutoff:
        dataset = TimeCutoffDataset(config)
    else:
        dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Define model
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])

    # Define trainer
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # Start training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # Start evaluating
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

## 4.2. Save config as yaml file

In [ ]:
with open(utils.get_path_conf(), 'w+') as f:
    yaml.dump(config_dict, f, allow_unicode=True)

## 4.2. Start tuning

In [ ]:
tuning_algo = "bayes"
early_stop = 10
max_evals = 10

hp = HyperTuning(
    objective_function=objective_function,
    algo=tuning_algo,
    early_stop=early_stop,
    max_evals=max_evals,
    params_dict=hyper_params,
    fixed_config_file_list=[utils.get_path_conf()]
)


hp.run()

running parameters:                                   
{'learning_rate': 0.0001}                             
current best valid score: 0.0476                      
current best valid result:                            
OrderedDict([('ndcg@10', 0.0476), ('precision@10', 0.01), ('recall@10', 0.0997), ('mrr@10', 0.0322), ('hit@10', 0.0997), ('map@10', 0.0322)])
current test result:                                  
OrderedDict([('ndcg@10', 0.0452), ('precision@10', 0.0087), ('recall@10', 0.087), ('mrr@10', 0.0324), ('hit@10', 0.087), ('map@10', 0.0324)])
running parameters:                                                  
{'learning_rate': 0.0001}                                            
running parameters:                                                  
{'learning_rate': 0.0005}                                            
 20%|██        | 2/10 [04:02<16:10, 121.35s/trial, best loss: -0.0476]


KeyboardInterrupt: 

## 4.3. Export tunning result

In [ ]:
# print best parameters
print('best params: ', hp.best_params)

# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

# export to JSON file
tune_result = {
    'best_params': hp.best_params,
    'best_result': hp.params2result[hp.params2str(hp.best_params)]
}
with open(utils.get_path_tune_log(), "w+") as f:
    json.dump(tune_result, f, indent=2, ensure_ascii=False)

best params:  {'learning_rate': 0.005}
best result: 
{'model': 'BPR', 'best_valid_score': 0.054, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.141), ('mrr@10', 0.054), ('ndcg@10', 0.0742), ('hit@10', 0.141), ('precision@10', 0.0141)]), 'test_result': OrderedDict([('recall@10', 0.1304), ('mrr@10', 0.0483), ('ndcg@10', 0.0673), ('hit@10', 0.1304), ('precision@10', 0.013)])}
